# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# First I am going to read in the Dataframe that I created in the WeatherPy file
cities_file_location = "../Weather/output_data/cities.csv"

weather_df = pd.read_csv(cities_file_location)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Country,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph)
0,0,San Cristobal,7.7669,-72.2250,VE,67.17,99,94,3.60
1,1,Lompoc,34.6391,-120.4579,US,78.13,75,100,6.91
2,2,Hobart,-42.8794,147.3294,AU,56.26,80,100,1.01
3,3,Nikolskoye,59.7035,30.7861,RU,62.02,71,10,3.56
4,4,Chui,-33.6971,-53.4616,UY,47.80,81,51,10.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Here I am configuring gmaps 
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# I am assigning the weight of the heat map to the humidity % of each city so I need to read this in from the dataframe
rating = weather_df["Humidity %"].astype(float)


# Create initial figure with gmaps
fig = gmaps.figure(center = (30,0), zoom_level=1.7)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, point_radius=3.3, max_intensity=150)
# Add heat layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# drop null rows
weather_df = weather_df.dropna()

In [6]:
# Filter out rows that are too hot or too cold 
# Only temps from 70 to 80 will still be in dataframe
not_too_cold_df = weather_df.loc[weather_df["Temperature (F)"]>=70]
ideal_temp_df = not_too_cold_df.loc[not_too_cold_df["Temperature (F)"]<=80]


# From this ideal temperature dataframe I am now going to filter out rows where it is too windy
# I will define too windy as any city that has wind speeds of over 10 mph 
ideal_wind_df = ideal_temp_df.loc[ideal_temp_df["Wind Speed (mph)"]<=10]

ideal_cloud_df = ideal_wind_df.loc[ideal_wind_df["Cloudiness %"]<=50]
ideal_weather_df = ideal_cloud_df.loc[ideal_cloud_df["Humidity %"]<=70]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a series of latitudes and longitudes and convert them to lists
lat = ideal_weather_df["Latitude"]
lat = lat.tolist()
longt = ideal_weather_df["Longitude"]
longt = longt.tolist()

In [8]:
# Set up parameters 
target_search = "Hotel"
target_radius = 5000
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up blank list 
hotels = []

for x in range(len(ideal_weather_df)):
    try:
        target_coordinates = str(lat[x]) +", "+ str(longt[x])
        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "key": g_key,
            "rankBy": "DISTANCE"}
        response = requests.get(base_url, params=params)
        hotel_data = response.json()
        hotels.append(hotel_data['results'][0]['name'])
    except IndexError:
        hotels.append("NA")

In [9]:
# Add hotels list to the dataframe and remove the rows where the dataframe did not find a value
ideal_weather_df["Hotels"] = hotels 

hotels_df = ideal_weather_df.loc[ideal_weather_df['Hotels']!="NA"]

<ipython-input-9-f8d896693612>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotels"] = hotels


In [10]:
# Add marker layer ontop of heat map
marker_locations = hotels_df[["Latitude", "Longitude"]]

# Add markers
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))